In [65]:
!pip install chromium
!pip install chromedriver_autoinstaller

ERROR: Could not find a version that satisfies the requirement chromium (from versions: none)
ERROR: No matching distribution found for chromium
You should consider upgrading via the 'C:\Users\Sveta\projects\недвижка парсер\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Sveta\projects\недвижка парсер\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [69]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import numpy as np
import requests
import csv
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from time import sleep

options = Options()
options.binary_location = r"C:\Program Files\Google\Chrome Beta\Application\chrome.exe"
# chrome = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver = webdriver.Chrome(options = options, executable_path='chromedriver.exe')

C:\Users\Sveta\AppData\Local\Temp\ipykernel_10904\2674266449.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options = options, executable_path='chromedriver.exe')


In [16]:
def get_html(url, PARAMS = None): 
    html = driver.page_source
    return html

def get_content(html): 
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('article', class_ = '_93444fe79c--container--Povoi')
    return items
    
def get_urls():   
    urls = []
    params = ['&sort=creation_date_desc', 
              '&sort=creation_date_asc', 
              '&sort=price_object_order', 
              '&sort=total_price_desc', 
              '&sort=street_name', 
              '&sort=walking_time']
    
    for par in tqdm(range(len(params))):
        param = params[par] 
        for i in range(1,55): 
            URL = driver.get(f'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p={i}&region=1&room1=1&{param}&type=4')
            html = get_html(URL)
            items = get_content(html)
            for item in items:
                urls.append(item.find('a', class_='_93444fe79c--link--eoxce').get('href'))
        urls = list(set(urls))
    return urls

In [5]:
def get_data(urls): 
    prices, metro, floor, max_floor, size, msize, deadline, bathroom, renovation, time = [], [], [], [], [], [], [], [], [], []
    for i in tqdm(range(len(urls))):
        url = urls[i]
        driver.set_page_load_timeout(0.8)
        URL = driver.get(url)
        html = get_html(URL)
        soup = BeautifulSoup(html, 'html.parser')
        try: 
            price = soup.find('span', class_ = "a10a3f92e9--price_value--lqIK0").find('span').get('content')
            prices.append(int(price.replace(' ', '').rstrip('₽/мес.')))
        except AttributeError: prices.append(np.nan)
            
        try: metro.append(soup.find('a', class_ = 'a10a3f92e9--underground_link--Sxo7K').get_text())
        except AttributeError: metro.append(np.nan)
           
        try: time.append(int(soup.find('span', class_ = 'a10a3f92e9--underground_time--iOoHy').get_text()[4:6]))
        except AttributeError: time.append(np.nan)
        except ValueError: time.append(np.nan)
         
        block1 = soup.find_all('div', class_ = 'a10a3f92e9--info-value--bm3DC')
        titles1 = soup.find_all('div', class_ = 'a10a3f92e9--info-title--JWtIm')
        
        flag = [0,0,0,0]
        for i in range(len(titles1)):
            if titles1[i].get_text() == 'Общая':
                flag[0] = 1
                try: size.append(float(block1[i].get_text().rstrip('\xa0м²').replace(',','.')))
                except ValueError: size.append(np.nan)
            elif titles1[i].get_text() == 'Жилая':
                flag[1] = 1
                try: msize.append(float(block1[i].get_text().rstrip('\xa0м²').replace(',','.')))
                except ValueError: msize.append(np.nan)
            elif titles1[i].get_text() == 'Этаж':
                flag[2] = 1
                floors = block1[i].get_text().split(' из ')
                floor.append(int(floors[0]))
                max_floor.append(int(floors[1]))
            elif titles1[i].get_text() == 'Построен' or titles1[i].get_text() == 'Срок сдачи':
                flag[3] = 1
                deadline.append(int(block1[i].get_text()[-4:]))
        if flag[0] == 0:
            size.append(np.nan)
        if flag[1] == 0:
             msize.append(np.nan)
        if flag[2] == 0:
            floor.append(np.nan)
            max_floor.append(np.nan)
        if flag[3] == 0:
            deadline.append(np.nan)

        block2 = soup.find_all('span', class_ = 'a10a3f92e9--value--Y34zN')
        titles2 = soup.find_all('span', class_ = 'a10a3f92e9--name--x7_lt')

        flag = [0,0,0,0]
        for i in range(len(titles2)):
            if titles2[i].get_text() == 'Санузел':
                flag[0] = 1
                bathroom.append(block2[i].get_text())
            elif titles2[i].get_text() == 'Ремонт':
                flag[1] = 1
                renovation.append(block2[i].get_text())
        if flag[0] == 0:
            bathroom.append(np.nan)
        if flag[1] == 0:
            renovation.append(np.nan)
            
        
    return prices, metro, size, msize, floor, max_floor, deadline, bathroom, renovation, time

In [ ]:
urls = get_urls()
prices, metro, size, msize, floor, max_floor, deadline, bathroom, renovation, time = get_data(urls)

In [87]:
df = pd.DataFrame({ 
    'Ссылка' : urls, 
    'Метро' : metro, 
    'Время до метро' : time,
    'Общая площадь' : size, 
    'Жилая площадь' : msize, 
    'Этаж' : floor,
    'Этажность дома' : max_floor,
    'Срок сдачи' : deadline,
    'Ванная' : bathroom,
    'Ремонт': renovation,
    'Цена' : prices
    })

In [88]:
df.to_csv('cian.csv')

# AVITO

In [50]:
def get_content_avito(html): 
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_ = 'iva-item-content-rejJg')
    return items
    
def get_urls_avito():   
    urls = []
    for i in range(100,101): 
        URL = driver.get(f'https://www.avito.ru/moskva/kvartiry/sdam/na_dlitelnyy_srok-ASgBAgICAkSSA8gQ8AeQUg?f=ASgBAgECAkSSA8gQ8AeQUgFFxpoMFXsiZnJvbSI6MCwidG8iOjYwMDAwfQ&p={i}')
        html = get_html(URL)
        items = get_content_avito(html)
        for item in items:
            urls.append(item.find('a', class_='iva-item-sliderLink-uLz1v').get('href'))
        urls = list(set(urls))
    return urls

In [51]:
urls_avito = get_urls_avito()
urls_avito

['/moskva/kvartiry/1-k._kvartira_35m_1012et._1625365412',
 '/moskva/kvartiry/1-k._kvartira_40m_1325et._2524972062',
 '/moskva/kvartiry/2-k._kvartira_53m_614et._2504567626',
 '/moskva/kvartiry/1-k._kvartira_331m_49et._2412522443',
 '/moskva/kvartiry/3-k._kvartira_68m_1117et._2504434527',
 '/moskva/kvartiry/1-k._kvartira_35m_25et._2324797358',
 '/moskva/kvartiry/2-k._kvartira_60m_1525et._2481342088',
 '/moskva/kvartiry/1-k._kvartira_40m_617et._2513744405',
 '/moskva/kvartiry/2-k._kvartira_74m_1118et._2527086931',
 '/moskva/kvartiry/2-k._kvartira_54m_1718et._2392099661',
 '/moskva/kvartiry/1-k._kvartira_43m_1222et._2522338799',
 '/moskva/kvartiry/2-k._kvartira_445m_1414et._2523602844',
 '/moskva/kvartiry/3-k._kvartira_60m_45et._1868108346',
 '/moskva/kvartiry/1-k._kvartira_47m_1014et._2481282616',
 '/moskva/kvartiry/1-k._kvartira_38m_517et._2512459861',
 '/moskva/kvartiry/3-k._kvartira_63m_1012et._2129022372',
 '/moskva/kvartiry/2-k._kvartira_578m_1223et._2449216930',
 '/moskva/kvartiry/3

In [52]:
len(urls_avito)

50

In [60]:
def get_data_avito(urls): 
    prices, metro, floor, max_floor, size, bathroom, renovation, time = [], [], [], [], [], [], [], []
    for i in tqdm(range(len(urls))):
        url = urls[i]
        URL = driver.get(url)
        sleep(5)
        html = get_html(URL)
        soup = BeautifulSoup(html, 'html.parser')
        try: 
            price = soup.find('span', class_ = "js-item-price style-item-price-text-2u_qK text-text-1PdBw text-size-xxl-1Uoae").find('span').get('content')
            prices.append(int(price.replace(' ', '').rstrip('₽/мес.')))
        except AttributeError: prices.append(np.nan)
            
        try: metro.append(soup.find('a', class_ = 'style-item-address-georeferences-item-18pFt').get_text())
        # 'geo-georeferences-Yd_m5 text-text-LurtD text-size-s-BxGpL'
        except AttributeError: metro.append(np.nan)
           
        try: time.append(int(soup.find('span', class_ = 'style-item-address-georeferences-item-interval-izti5').get_text()[4:6]))
        except AttributeError: time.append(np.nan)
        except ValueError: time.append(np.nan)
         
        block1 = soup.find_all('text')
        titles1 = soup.find_all('span', class_ = 'css-1r29yiv')
        
        flag = [0,0,0,0]
        for i in range(len(titles1)):
            if titles1[i].get_text() == 'Общая':
                flag[0] = 1
                try: size.append(float(block1[i].get_text().rstrip('\xa0м²').replace(',','.')))
                except ValueError: size.append(np.nan)
            elif titles1[i].get_text() == 'Этаж':
                flag[1] = 1
                floors = block1[i].get_text().split(' из ')
                floor.append(int(floors[0]))
                max_floor.append(int(floors[1]))
        if flag[0] == 0:
            size.append(np.nan)
        if flag[1] == 0:
            max_floor.append(np.nan)

        block2 = soup.find_all('text')
        titles2 = soup.find_all('span', class_ = 'css-1r29yiv')

        flag = [0,0,0,0]
        for i in range(len(titles2)):
            if titles2[i].get_text() == 'Санузел':
                flag[0] = 1
                bathroom.append(block2[i].get_text())
            elif titles2[i].get_text() == 'Ремонт':
                flag[1] = 1
                renovation.append(block2[i].get_text())
        if flag[0] == 0:
            bathroom.append(np.nan)
        if flag[1] == 0:
            renovation.append(np.nan)
        driver.switch_to.window(driver.window_handles[1])
        
    return prices, metro, size, floor, max_floor, bathroom, renovation, time

In [71]:
if __name__ == 'get_data_avito':
    try:
        service = Service(executable_path=ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        prices, metro, size, floor, max_floor, bathroom, renovation, time = get_data_avito(urls_avito)
    except Exception as e:
        print(e)
    finally:
        driver.quit()

In [70]:
prices, metro, size, floor, max_floor, bathroom, renovation, time = get_data_avito(urls_avito)


  0%|          | 0/50 [00:00<?, ?it/s]


InvalidArgumentException: Message: invalid argument
  (Session info: chrome=105.0.5195.52)
Stacktrace:
Backtrace:
	Ordinal0 [0x00F78823+2197539]
	Ordinal0 [0x00F10911+1771793]
	Ordinal0 [0x00E23EB0+802480]
	Ordinal0 [0x00E173C6+750534]
	Ordinal0 [0x00E15ED5+745173]
	Ordinal0 [0x00E1619D+745885]
	Ordinal0 [0x00E257B4+808884]
	Ordinal0 [0x00E7E12D+1171757]
	Ordinal0 [0x00E6D65C+1103452]
	Ordinal0 [0x00E7DA72+1170034]
	Ordinal0 [0x00E6D476+1102966]
	Ordinal0 [0x00E47660+947808]
	Ordinal0 [0x00E48556+951638]
	GetHandleVerifier [0x01221A32+2740578]
	GetHandleVerifier [0x01213776+2682534]
	GetHandleVerifier [0x0100307A+518058]
	GetHandleVerifier [0x01002196+514246]
	Ordinal0 [0x00F17ADB+1800923]
	Ordinal0 [0x00F1C2E8+1819368]
	Ordinal0 [0x00F1C3D5+1819605]
	Ordinal0 [0x00F25810+1857552]
	BaseThreadInitThunk [0x76326739+25]
	RtlGetFullPathName_UEx [0x770F90AF+1215]
	RtlGetFullPathName_UEx [0x770F907D+1165]


In [ ]:
df = pd.DataFrame({ 
    'Ссылка' : urls, 
    'Метро' : metro, 
    'Время до метро' : time,
    'Общая площадь' : size, 
    'Этаж' : floor,
    'Этажность дома' : max_floor,
    'Ванная' : bathroom,
    'Ремонт': renovation,
    'Цена' : prices
    })

df.to_csv('avito.csv')

In [44]:
def extract_data_avito(flat_url):
    '''
    применяется к каждой отдельной ссылке
    на страницу квартиры
    '''
    html = get_html(flat_url)
    soup = BeautifulSoup(html, features='html.parser')
    row = []
    
    # price
    try:
        row.append(int(soup.find('span', class_='s-item-price style-item-price-text-2u_qK text-text-1PdBw text-size-xxl-1Uoae').get('content')))
    except AttributeError:
        row.append(None)

    
    # from uls
    area = None
    rooms = None
    floors = None
    house_type = None
    constr_date = None
    ceiling_height = None

    lis = soup.find_all('li', class_='item-params-list-item')
    for li in lis:

        # area, rooms, floors --- about flat
        if li.find('span', class_='item-params-label').get_text() == 'Общая площадь: ':
            area = str(li.contents[2]).split()[0]
        if li.find('span', class_='item-params-label').get_text() == 'Количество комнат: ':
            rooms = str(li.contents[2]).split()[0]
        if li.find('span', class_='item-params-label').get_text() == 'Этаж: ':
            floors = ' '.join(str(li.contents[2]).split())

        # house type, date of construction
        if li.find('span', class_='item-params-label').get_text() == 'Тип дома: ':
            house_type = str(li.contents[2]).split()[0]
        if li.find('span', class_='item-params-label').get_text() == 'Год постройки: ':
            constr_date = str(li.contents[2]).split()[0]
        if li.find('span', class_='item-params-label').get_text() == 'Срок сдачи: ':
            constr_date = ' '.join(str(li.contents[2]).split())
        if li.find('span', class_='item-params-label').get_text() == 'Высота потолков: ':
            ceiling_height = str(li.contents[2]).split()[0]

    for i in [area, rooms, floors, house_type, constr_date, ceiling_height, flat_url]:
        row.append(i)

    return row

In [45]:
data_avito = extract_data_avito(urls_avito)
data_avito[:5]

[None, None, None, None, None]